# **들어가기에 앞서**

**Tabular Playground Series(이하 tabular)는 데이터 분석에 입문한 초보자들이 데이터를 다뤄보고 예측해보는 것을 목표로 한 대회입니다.**

**저도 초보자로써 tabular대회를 접하였으나 여러가지 방법을 시도하여 5월 대회 leader board 14위를 하였고**

**이 과정에서 처음 입문한 사람들이 데이터를 학습시키고 submission을 제출해보는 가이드라인이 필요하다고 느꼈습니다.**

**kaggle에서 자주 쓰는 lightgbm과 random forest만을 다뤘습니다.**

**저도 초보인만큼 "어? 이거 아닌데" 하는 부분이 많습니다. 질문과 지적은 환영입니다.**

-------

**tabular대회 특성상 feature간의 연관성도 적고 데이터 전처리에 대한 부분이 적다고 판단했기에 eda 부분은 다른 분들의 노트북을 보는 것을 추천드립니다**

**실제 데이터 분석에서는 데이터 전처리가 굉장히 중요하기 때문에 데이터 전처리를 익히기 위해서는 다른 competition에 참가하는 것을 추천드립니다**

----------

**train dataset은 학습을 위해 각 feature의 값과 실제값이 있는 dataset입니다.**

**test dataset은 예측을 위한 dataset으로 feature 값이 있고 이를 토대로 target 값을 예측해야 합니다.**

**sample_submission은 제출해야 하는 submission의 형식을 보여줍니다.**

----------

# **라이브러리 import**

**사용하게 될 라이브러리를 import 하는 과정입니다.**

**수학적 연산을 도와주는 numpy,**

**데이터 처리를 도와주는 pandas를 기본적으로 import 해줍니다.**

In [ ]:
import numpy as np
import pandas as pd

**그리고 불필요한 경고로 스크롤이 늘어나는 것을 대비하여**

**warnings를 import하여 경고를 무시해줍니다.**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# **데이터 로드**

**데이터를 보고 처리하기 위하여 pandas dataframe 형식으로 csv파일을 읽습니다.**

**pandas의 read_csv에 경로를 입력하여 데이터를 받아옵니다.** 

**경로는 input에 있는 데이터에서 copy path를 하면 쉽게 얻을 수 있습니다.**

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
#train 변수에 dataframe형식으로 읽어온 train.csv파일을 저장합니다.

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
#test 변수에 dataframe형식으로 읽어온 test.csv파일을 저장합니다.

In [ ]:
sample = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")
#test 변수에 dataframe형식으로 읽어온 sample_submission.csv파일을 저장합니다.

# **데이터 구경**

**pandas dataframe형식을 지원하는 .head()와 shape기능을 사용하여 데이터가 어떤 형식인지 조금이라도 구경합니다.**

In [ ]:
train.shape
#학습을 위한 train data는 250000개의 행과 102개의 column으로 이루어져 있다고 합니다.

In [ ]:
test.shape
#예측 해야되는 test data는 150000개의 행이 있고 예측해야할 값인 loss column을 제외한 101개의 column으로 이루어져 있습니다.

In [ ]:
train.head(10)
#pandas dataframe 형식은 .head()를 사용하여 데이터를 위에서부터 볼 수 있습니다.

In [ ]:
test.head()
#dataframe형식이면 head()를 이용하여 쉽게 값을 볼 수 있고 default값은 5개입니다

In [ ]:
sample.head()
#제출해야하는 데이터 형식을 볼 수 있습니다.

# **머신러닝 모델 적용**

**훈련데이터를 나누고 valid dataset으로 검증하는 과정을 설명합니다.**

**모델을 구현하기보다 RandomForestRegressor, LGBMRegressor를 import하여 사용하였습니다.**

**그러나 좋은 성적을 위해서는 모델의 작동원리에 대한 이해가 필요합니다.**

In [ ]:
X = train.drop(["loss","id"],axis=1)
y = train["loss"]
#들어가기에 앞서 훈련데이터를 학습에 사용할 feature와 그에 대한 target 값으로 나눕니다.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=38)
#overfitting과 underfitting등의 문제를 확인하기 위해 valid dataset를 만듭니다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor()
random_forest.fit(X_train,y_train)

In [ ]:
from lightgbm import LGBMRegressor
lightgbm = LGBMRegressor()
lightgbm.fit(X_train,y_train)

# **모델 평가**

**valid dataset을 이용하여 모델의 성능을 평가합니다.**

**실제 대회에서 사용되는 root_mean_squared_error 평가 지표를 사용하였습니다.**

In [ ]:
from sklearn.metrics import mean_squared_error
#sklearn에서 mean_squared_error 평가 지표를 import합니다

In [ ]:
def rmse(y_true, y_pred):
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    return sqrt(mean_squared_error(y_true, y_pred))
#원하는 값은 root를 적용한 값이기 때문에 math 라이브러리에서 root를 이용하여 값을 리턴하도록 합니다

**함수에 예측값과 실제값을 넣어 metrics를 계산한 score를 출력합니다.**

In [ ]:
random_forest_pred=random_forest.predict(X_val)
random_forest_score = rmse(random_forest_pred,y_val)
print("random forest 알고리즘의 점수:",random_forest_score)

In [ ]:
lightgbm_pred=lightgbm.predict(X_val)
lightgbm_score = rmse(lightgbm_pred,y_val)
print("lightgbm 알고리즘의 점수:",lightgbm_score)

# **성능 향상을 위한 약간의 하이퍼 파라미터 튜닝**

**모델의 성능을 극대화 시키기 위한 방법은 여러가지가 있지만**

**보편적으로 모델의 하이퍼 파라미터를 설정하여 데이터에 맞는 파라미터 값을 찾습니다.**

In [ ]:
from sklearn.model_selection import GridSearchCV

**GridSearchCV는 지정된 모든 경우를 테스트하고 최적의 파라미터를 찾아줍니다.**

**for문을 이용해서도 구현이 가능한 직관적인 알고리즘입니다.**

In [ ]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor()

lgbm_param = {"learning_rate" : [0.1, 0.038,0.003, 0.001]}
lgbm_grid_search = GridSearchCV(lgbm,param_grid=lgbm_param)

lgbm_grid_search.fit(X_train,y_train)
y_pred=lgbm_grid_search.predict(X_val)
lgbm_score = rmse(y_pred,y_val)
print(lgbm_score)
print(lgbm_grid_search.best_params_)

**작동구조를 이해하기 위하여 for문으로 구현을 하자면 list에 실험할 파라미터 값이 있을 때,**

**그 list의 값을 파라미터에 적용하여 나온 결과값을 토대로 최적의 파라미터 값을 찾습니다.**

In [ ]:
lgbm_lr = [0.1, 0.038,0.003, 0.001]

In [ ]:
min_score=100
for i in lgbm_lr:
    lgbm_grid_search = LGBMRegressor(learning_rate=i)
    lgbm_grid_search.fit(X_train,y_train)
    y_pred=lgbm_grid_search.predict(X_val)
    score = rmse(y_pred,y_val)
    if score<min_score:
        min_score=score
        min_lr=i
    print("lr = ",i," valid_score = ",score)
            
print("--------------------------------")
print("min_lr = ",min_lr," min_valid_score = ",min_score)

**이해를 돕기위하여 간단한 파라미터만 조정해보았습니다.**

**다양한 파라미터를 조정하여 좋은 인공지능 모델을 찾을 수 있었으면 좋겠습니다.**

# **submission 만들기**

**sample_submission의 id와 test_data의 예측값을 이용하여 결과를 csv파일로 만듭니다.**

**pandas의 to_csv 기능을 이용하면 dataframe형태의 값을 csv형식으로 변환 할 수 있습니다.**

In [ ]:
ids = sample["id"]
#sample_submission의 id 값을 이용합니다

In [ ]:
preds = random_forest.predict(test.drop('id', axis=1))
output = pd.DataFrame({"id":ids, "loss":preds})
output.to_csv("random_forest.csv", index=False)

In [ ]:
preds = lightgbm.predict(test.drop('id', axis=1))
output = pd.DataFrame({"id":ids, "loss":preds})
output.to_csv("lightgbm.csv", index=False)

In [ ]:
preds = lgbm_grid_search.predict(test.drop('id', axis=1))
output = pd.DataFrame({"id":ids, "loss":preds})
output.to_csv("lgbm_grid_search.csv", index=False)

# **모델의 성능을 높이는 방법**

**모델의 하이퍼 파라미터 튜닝 말고도 모델의 성능을 높이기 위한 여러가지 방법이 있습니다.**

**1. feature engineering을 진행한다.**

**2. model 앙상블 등 모델을 합쳐 더 좋은 성능을 낸다.**

**3. 더 많은 데이터를 이용하기 위해 pseudo labeling등의 방법을 사용한다.**

**여러 방법들을 이용하여 좋은 성적을 거두었으면 좋겠습니다.**